# Ejecutar un Job

Es posible trabajar en DevCloud o en cualquier otro cluster de alta performancia usando el modo _interactivo_. Sin embargo, algunas veces los projectos en los que trabajamos requieren muchos recursos que no están disponibles en un momento dado. Para ello podemos usar modo _batch_, en el cual uno puede iniciar la ejecucion de **Jobs** para completar tareas. Esto puede pensarse como hacer una cola en el banco y esperar tu turno para hacer tu trámite. Hacemos nuestra cola con la tarea a ser ejecutada y esperamos hasta que los recursos sean liberados.

![alt text](job_1.png)

## Ejemplo de Job 1: Hello World en C

- `writefile` permite escribir y guardar un código fuente usando la interfaz de Jupyter Notebooks. Dado que queremos escribir un archivo fuente en C++, vamos a colocar luego de `writefile` el nombre de nuestro archivo con la extensión *.cpp*.
- Al inicio del archivo añadimos `#include <stdio.h>` para indicar al compilador que inserte el contenido de la librería estandar `stdio` en el archivo fuente que estamos creando.
- Finalmente escribimos el cuerpo del programa *hello world* dentro de la funcion `int main()`. Esta función debe retornar un valor entero (`int` o _integer_) al final de la ejecución de la función `main`. Por ello, como buena práctica terminamos el código con `return 0;`. La función `printf` viene del término en inglés imprimir con formato (_"print formatted"_) e imprime un conjunto de caracteres (o _string_) en C++. Para mayor información, visitar el siguiente [enlace](https://cplusplus.com/reference/cstdio/printf/).

In [1]:
%%writefile hello_A.cpp

#include <stdio.h>

int main()
{
    printf("Hello World");

    return 0;
}

Writing hello_A.cpp


Hemos producido el archivo fuente **hello_A.cpp**. Ahora debemos compilarlo (porque es un archivo C++ y al compilarlo producimos el archivo binario o ejecutable) y ejecutarlo (se ejecuta el archivo binario o ejecutable). Vamos a usar el compilador de Intel llamado `dpcpp`. Para ello creamos un nuevo archivo con extensión *.pbs*, el cual nos va permitir iniciar la ejecucion del **Job**. En otras palabras, este archivo detalla los recursos que requerimos para compilar y ejecutar **hello_A.cpp**. 

En la siguiente figura vamos a ver el detalle de los componentes de un archivo bash para ejecutar un job:

![alt text](job_2.png)

**Qué hace y cómo funciona el compilador `dpcpp`**:

El `dpcpp` crea el archivo binario o ejecutable. Luego de colocar el comando `dpcpp`, le sige el primer flag `-o` que nos indica que lo que sigue el nombre que le vamos a dar al archivo binario y la ubicación. En este caso, nosotros le llamaremos *hello_A* y lo almacenaremos en nuestra carpeta de trabajo. Podemos colocar la ubicación absoluta usando `pwd` en el terminal o la relativa usando `HOME`. Seguidamente, tenemos la la ruta dónde se encuentra el archivo fuente *.cpp* (absoluta o relativa). Finalmente, ejecutamos el archivo binario recientemente creado *hello_A*, sin olvidar dar su ubicación. 

In [6]:
%%writefile compi_A.pbs
#PBS -S /bin/bash
#
#PBS -N hello
#PBS -l nodes=1,walltime=00:05:00
#
dpcpp -o $HOME/CPAR-INTRO/hello_A $HOME/CPAR-INTRO/hello_A.cpp

$HOME/CPAR-INTRO/hello_A

Overwriting compi_A.pbs


Para poder realizar la tarea explicada en *compi_A.pbs*, vamos a usar el comando `qsub` que nos indica que estamos subiendo un job. Si se ejecuta directamente en Jupyter Lab y no en el terminal de DevCloud, se debe iniciar usar `!qsub`: 

In [7]:
!qsub compi_A.pbs

2339933.v-qsvr-1.aidevcloud


Para ver el status del job, usar `qstat`. El primer resultado es un job interactivo donde estamos ejecutando jupyter lab, y el segundo es nuestro archivo denominado hello3: 

In [9]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2339831.v-qsvr-1           ...ub-singleuser u196481         00:00:29 R jupyterhub     
2339933.v-qsvr-1           hello            u196481                0 R batch          


El resultado del job se visualiza en el archivo de extensión *.oxxxx*: 

![alt text](job_3.png)

Cualquier error que uno hay encontrado se visualiza en el archivo de extensión *.exxxx*:

![alt text](job_4.png)

## Ejemplo de Job2: Curva seno en python

Del mismo modo que *Hello World* creamos un archivo esta vez de extensión *.py*. En este archivo lo que queremos es crear la curva seno y plotearla. Para ello cargamos las librerías *numpy* y *matplotlib*. Luego creamos un vector x y aplicamos la función *seno*. Finalmente, plotamos y guardamos el archivo con el nombre sen_curva

In [28]:
%%writefile sen_curva.py

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

x = np.arange(2*np.pi,10*np.pi,0.1)   # inicio,final,intervalo
y = np.sin(x)

fig = plt.figure()
plt.plot(x,y)
plt.savefig("/home/u196481/CPAR-INTRO/sen_curva.png", dpi = 72)
plt.close(fig)

Writing sen_curva.py


Luego creamos el archivo que contendrá el job que queremos subir. La diferencia con el archivo anterior es que aquí es que estamos abriendo **python** y dentro de **python**, ejecutando el archivo **sen_curva**. Nota: No olvidar la ubicación del archivo. 

In [29]:
%%writefile compi_py.pbs
#PBS -S /bin/bash
#
#PBS -N curva_seno
#PBS -l nodes=1,walltime=00:05:00
#

python /home/u196481/CPAR-INTRO/sen_curva.py

Writing compi_py.pbs


In [30]:
!qsub compi_py.pbs

2339955.v-qsvr-1.aidevcloud


In [31]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2339831.v-qsvr-1           ...ub-singleuser u196481         00:00:44 R jupyterhub     
2339955.v-qsvr-1           curva_seno       u196481                0 R batch          


Si todo es correcto, tendremos un archivo de extensión .*png* cómo resultado:

![alt text](job_5.png)